In [38]:
import pandas as pd
import numpy as np
from prophet import Prophet
import plotly.graph_objects as go
import plotly.io as pio

# Configure Plotly to open in browser
pio.renderers.default = 'browser'

In [39]:
df = pd.read_csv("Created Dataset/global_daily_cases_deaths.csv")

In [40]:
df.head()

,Date,Daily_Cases,Daily_Deaths
0,2020-01-23,0.0,0.0
1,2020-01-24,187.0,7.0
2,2020-01-25,206.0,8.0
3,2020-01-26,190.0,0.0
4,2020-01-27,126.0,12.0


In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1142 entries, 0 to 1141
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          1142 non-null   object 
 1   Daily_Cases   1142 non-null   float64
 2   Daily_Deaths  1142 non-null   float64
dtypes: float64(2), object(1)
memory usage: 26.9+ KB


In [42]:
df['Date'] = pd.to_datetime(df['Date'])

In [43]:
# Prepare data for Prophet
cases_df = df[['Date', 'Daily_Cases']].rename(columns={'Date': 'ds', 'Daily_Cases': 'y'})
deaths_df = df[['Date', 'Daily_Deaths']].rename(columns={'Date': 'ds', 'Daily_Deaths': 'y'})

In [44]:
# Apply log-transform (to handle spikes and avoid negative predictions)
cases_df['y'] = np.log1p(cases_df['y'])
deaths_df['y'] = np.log1p(deaths_df['y'])

In [45]:
# Build Prophet models
cases_model = Prophet(daily_seasonality=True)
cases_model.fit(cases_df)

deaths_model = Prophet(daily_seasonality=True)
deaths_model.fit(deaths_df)

13:55:52 - cmdstanpy - INFO - Chain [1] start processing
13:55:53 - cmdstanpy - INFO - Chain [1] done processing
13:55:53 - cmdstanpy - INFO - Chain [1] start processing
13:55:53 - cmdstanpy - INFO - Chain [1] done processing


In [46]:
# Make future dataframe (30 days)
future_cases = cases_model.make_future_dataframe(periods=30)
future_deaths = deaths_model.make_future_dataframe(periods=30)


In [47]:
# Predict next 30 days
cases_forecast = cases_model.predict(future_cases)
deaths_forecast = deaths_model.predict(future_deaths)

In [48]:
# Inverse log-transform to get real counts
cases_forecast['yhat'] = np.expm1(cases_forecast['yhat'])
deaths_forecast['yhat'] = np.expm1(deaths_forecast['yhat'])

In [49]:
# Extract only the next 30 days
pred_cases_30 = cases_forecast[['ds', 'yhat']].tail(30).rename(columns={'yhat': 'Predicted_Daily_Cases'})
pred_deaths_30 = deaths_forecast[['ds', 'yhat']].tail(30).rename(columns={'yhat': 'Predicted_Daily_Deaths'})

In [52]:
# Combine into one dataframe
pred_30_df = pd.merge(pred_cases_30, pred_deaths_30, on='ds').reset_index(drop=True)
print(pred_30_df.head())

          ds  Predicted_Daily_Cases  Predicted_Daily_Deaths
0 2023-03-10              71.515258                7.278924
1 2023-03-11               0.440338                0.874921
2 2023-03-12               0.934927                1.723636
3 2023-03-13            4779.117666              658.984322
4 2023-03-14            4883.542354             1069.182565


In [51]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=pred_30_df['ds'],
    y=pred_30_df['Predicted_Daily_Cases'],
    mode='lines+markers',
    name='Predicted Daily Cases',
    line=dict(color='crimson')
))

fig.add_trace(go.Scatter(
    x=pred_30_df['ds'],
    y=pred_30_df['Predicted_Daily_Deaths'],
    mode='lines+markers',
    name='Predicted Daily Deaths',
    line=dict(color='gray')
))

fig.update_layout(
    title="🌍 Global COVID-19: Predicted Daily Cases and Deaths (Next 30 Days)",
    xaxis_title="Date",
    yaxis_title="Count",
    template="plotly_white"
)

fig.show()